# DistilBERT로 자연어 추론(Natural Language Inference) 구현하기

이번 실습에서는 pre-trained된 DistilBERT를 불러와 MNLI(Multi-genre Natural Language Inference) 데이터셋에 적용합니다. MNLI 과제는 두 문장(premise와 hypothesis)의 관계를 예측하는 문제입니다.

In [1]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses datasets pandas

필요한 라이브러리를 불러오고 데이터를 준비합니다. DistilBERT 모델을 위한 토크나이저도 불러옵니다.

In [ ]:
import torch
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import random

# 경로 설정 (필요시 수정)
path = '/Users/semyungpark/Documents/homework/data/MNLI'

# MNLI 데이터셋 로드 함수
def load_data(path, nrows=None):
    df = pd.read_csv(path, nrows=nrows, keep_default_na=False)
    data = []
    for _, row in df.iterrows():
        if len(row['premise']) * len(row['hypothesis']) != 0:
            data.append({'premise': row['premise'], 'hypothesis': row['hypothesis'], 'label': row['label']})
    return data

# 데이터 로드 (각각 1000개의 예시만 사용)
train_data = load_data(path + '/train.csv', nrows=1000)
test_data = load_data(path + '/validation_matched.csv', nrows=1000)

# 토크나이저 로드 
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'distilbert-base-uncased')

Using cache found in /Users/semyungpark/.cache/torch/hub/huggingface_pytorch-transformers_main


# 데이터셋과 DataLoader 구현

MNLI 데이터에서 두 문장(premise, hypothesis)을 모델에 입력하기 위해서는 특별한 처리가 필요합니다. 
두 문장을 적절하게 토큰화하고 통합하여 모델에 제공해야 합니다.

In [ ]:
# MNLI 데이터셋을 위한 사용자 정의 Dataset 클래스
class MNLIDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        return item

# 데이터셋 객체 생성
train_dataset = MNLIDataset(train_data)
test_dataset = MNLIDataset(test_data)

# collate_fn 함수 정의: 배치 내의 샘플들을 처리
def collate_fn(batch):
    premises = [item['premise'] for item in batch]
    hypotheses = [item['hypothesis'] for item in batch]
    labels = [item['label'] for item in batch]
    
    # premise와 hypothesis를 [SEP] 토큰으로 구분하여 하나의 시퀀스로 결합
    # DistilBERT의 입력 형식: [CLS] premise [SEP] hypothesis [SEP]
    inputs = tokenizer(premises, hypotheses, padding=True, truncation=True, return_tensors='pt')
    
    # 라벨을 텐서로 변환 (MNLI는 3개의 클래스: entailment, contradiction, neutral)
    labels = torch.LongTensor(labels)
    
    return inputs, labels

# DataLoader 생성
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

Generating test split: 100%|██████████| 7600/7600 [00:00<00:00, 2380813.38 examples/s]



# MNLI 데이터셋 내용 확인하기

데이터셋의 구조와 실제 샘플을 확인해보겠습니다.

In [ ]:
# 데이터 기본 정보 출력
print(f"학습 데이터 샘플 수: {len(train_data)}")
print(f"테스트 데이터 샘플 수: {len(test_data)}")

# MNLI 레이블 정의 (0: entailment, 1: neutral, 2: contradiction)
label_names = {0: "entailment", 1: "neutral", 2: "contradiction"}
print("\n레이블 정의:")
for idx, name in label_names.items():
    print(f"{idx}: {name}")

# 샘플 데이터 확인
print("\n샘플 데이터:")
for i in range(3):
    sample = train_data[i]
    print(f"\n샘플 {i+1}:")
    print(f"전제(Premise): {sample['premise'][:100]}...")
    print(f"가설(Hypothesis): {sample['hypothesis'][:100]}...")
    print(f"레이블: {sample['label']} ({label_names[sample['label']]})")

# 클래스 분포 확인하기

데이터셋에서 각 클래스의 분포를 확인해보겠습니다.

In [ ]:
# 레이블 분포 확인
label_counts = {0: 0, 1: 0, 2: 0}
for sample in train_data:
    label_counts[sample['label']] += 1

# 결과 출력
print("클래스 분포:")
for label, count in label_counts.items():
    print(f"{label_names[label]}: {count} 샘플 ({count/len(train_data)*100:.1f}%)")

# 시각화
plt.figure(figsize=(10, 6))
plt.bar([label_names[i] for i in range(3)], [label_counts[i] for i in range(3)])
plt.title('MNLI 데이터셋 클래스 분포')
plt.ylabel('샘플 수')
plt.savefig('mnli_class_distribution.png')
plt.show()

# DistilBERT 모델 불러오기

사전학습된 DistilBERT 모델을 불러와서 자연어 추론 과제를 위한 분류 모델을 구현합니다.

In [ ]:
# DistilBERT 모델 로드
base_model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')

# 자연어 추론(NLI)을 위한 모델 정의
class NLIClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        
        # 사전학습된 DistilBERT 모델을 인코더로 사용
        self.encoder = base_model
        
        # 3개의 클래스(entailment, neutral, contradiction)를 분류하는 분류기
        self.classifier = nn.Linear(768, 3)
        
    def forward(self, inputs):
        # 토크나이저의 출력을 인코더에 전달
        outputs = self.encoder(inputs['input_ids'], 
                             attention_mask=inputs['attention_mask'])
        
        # [CLS] 토큰의 출력을 분류에 사용 (시퀀스의 첫 번째 토큰)
        pooled_output = outputs['last_hidden_state'][:, 0]
        
        # 분류기를 통과시켜 로짓 출력
        return self.classifier(pooled_output)

# 모델 초기화
model = NLIClassifier()

Using cache found in /Users/semyungpark/.cache/torch/hub/huggingface_pytorch-transformers_main
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Using cache found in /Users/semyungpark/.cache/torch/hub/huggingface_pytorch-transformers_main
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

# 모델 학습 준비

효율적인 학습을 위해 DistilBERT 인코더 부분은 고정(freeze)하고, 분류기 부분만 학습합니다.

In [ ]:
# DistilBERT 인코더 부분 고정 (파라미터 고정)
for param in model.encoder.parameters():
    param.requires_grad = False

# 학습 가능한 파라미터 수 확인
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"학습 가능한 파라미터: {trainable_params:,} ({trainable_params / total_params:.2%})")
print(f"전체 파라미터: {total_params:,}")

# 모델 학습 및 평가

이제 모델을 학습하고 성능을 평가합니다. M3 맥북을 위해 MPS 가속을 사용합니다.

In [ ]:
# 학습 설정
from torch.optim import Adam

# M3 맥북에서 MPS 디바이스 사용 설정
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(f"사용 디바이스: {device}")

model = model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=3e-5)
n_epochs = 5

# 학습 결과 저장을 위한 변수
train_losses = []
train_accs = []
test_accs = []

# 정확도 계산 함수
def compute_accuracy(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in data_loader:
            # 입력을 디바이스로 이동
            for key in inputs:
                inputs[key] = inputs[key].to(device)
            labels = labels.to(device)
            
            # 모델 예측
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            
            # 정확도 계산
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    return correct / total

# 학습 루프
for epoch in range(n_epochs):
    model.train()
    epoch_loss = 0.0
    
    for inputs, labels in train_loader:
        # 입력을 디바이스로 이동
        for key in inputs:
            inputs[key] = inputs[key].to(device)
        labels = labels.to(device)
        
        # 순전파
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        
        # 역전파 및 가중치 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    # 에포크별 손실 및 정확도 기록
    avg_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_loss)
    
    # 훈련 및 테스트 정확도 계산
    train_acc = compute_accuracy(model, train_loader)
    test_acc = compute_accuracy(model, test_loader)
    train_accs.append(train_acc)
    test_accs.append(test_acc)
    
    print(f"Epoch {epoch+1}/{n_epochs} | Loss: {avg_loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")

# 학습 결과 시각화

손실 곡선과 정확도 변화를 시각화하여 모델의 학습 과정을 분석합니다.

In [ ]:
# 학습 손실 곡선 그래프 그리기
plt.figure(figsize=(12, 5))

# 손실 곡선 그래프
plt.subplot(1, 2, 1)
plt.plot(range(1, n_epochs+1), train_losses, marker='o', linestyle='-', color='blue')
plt.title('Training Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)

# 정확도 변화 그래프
plt.subplot(1, 2, 2)
plt.plot(range(1, n_epochs+1), train_accs, marker='o', linestyle='-', color='blue', label='Train Accuracy')
plt.plot(range(1, n_epochs+1), test_accs, marker='o', linestyle='-', color='red', label='Test Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('mnli_training_results.png')
plt.show()

# 최종 성능 출력
print(f"최종 학습 정확도: {train_accs[-1]:.4f}")
print(f"최종 테스트 정확도: {test_accs[-1]:.4f}")

# 모델 성능 분석

클래스별 성능을 분석하여 모델이 어떤 유형의 추론에 강점과 약점을 가지는지 파악합니다.

In [ ]:
# 혼동 행렬(Confusion Matrix) 계산
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

def get_predictions(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in data_loader:
            # 입력을 디바이스로 이동
            for key in inputs:
                inputs[key] = inputs[key].to(device)
            
            # 예측
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            
            # 예측과 라벨 저장
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())
    
    return all_preds, all_labels

# 예측 결과 얻기
y_pred, y_true = get_predictions(model, test_loader)

# 혼동 행렬 생성 및 시각화
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=list(label_names.values()), 
            yticklabels=list(label_names.values()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig('mnli_confusion_matrix.png')
plt.show()

# 분류 보고서 출력
report = classification_report(y_true, y_pred, target_names=list(label_names.values()))
print("분류 보고서:")
print(report)